- 말뭉치(코퍼스)를 사용해 학습한 언어 모델을 이용해 새로운 문장을 생성

## 언어 모델을 사용한 문장 생성
### RNN을 사용한 문장 생성 순서
- 언어 모델 : 현재까지 주어진 단어들에서 다음에 출현하는 단의 확률분포를 출력
- 언어 모델로 선택된 단어를 선택하는 방법
  - 1. 확률이 가장 높은 단어를 선택(결과가 일정하게 정해짐)
  - 2. 각 후보 단어의 확률에 맞게 선택하는 것(결과가 다르게 정해질 수 있음)

- 생성 문장의 다양성을 위해서 확률적으로 선택하는 방법으로 구현 연습

- 언어 모델을 통해 생성한 문장은 훈련 데이터에 존재하지 않고, 새로 생성된 문장임
  - 언어 모델은 훈련 데이터를 암기하는 것이 아닌 사용된 단어의 정렬 패턴을 학습한 것임

### 문장 생성 구현


In [17]:
# 구글 드라이브에 있는 사용자 정의 모듈을  colab에서 사용하기 위한 사용자 설정
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/deep_learning2/

Mounted at /content/drive


In [3]:
# Rnnlm 클래스를 상속받아 RnnlmGen 클래스 생성
import numpy as np

from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm) : # Rnnlm을 상속받음

  # 문장을 생성하는 메서드
  # ========== 입력 파라미터 ========== 
  # start_id - 최초로 주는 단어의 ID
  # sample_size - 샘플링하는 단어의 수
  # skip_ids - 단어 Id의 리스트로 이에 속한 단어 ID는 샘플링하지 않음. PTB에 있는 <unk>나 전처리된 단어를 샘플링하지 않는 용도
  def generate(self, start_id, skip_ids=None, sample_size=100) :
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size : 
      x = np.array(x).reshape(1,1) # model.predict()에서 미니배치 처리하므로 2차원 배열로 성형(reshape)
      score = self.predict(x) # 각 단어의 정규화되기 전 점수(score) 출력
      p = softmax(score.flatten()) # 소프트맥스 함수로 정규화하여 확률분포 P를 얻음

      sampled = np.radnom.choice(len(p), size=1, p=p) # 확률 분포를 바탕으로 다음 단어 샘플링
      if (skip_ids is None) or (sampled not in skip_ids) :
        x = sampled
        word_ids.append(int(x))

    return word_ids


In [6]:
# 위에서 구현한 RnnlmGen 클래스를 사용하여 문장 생성
# 아무런 학습도 수행하지 않은 상태(가중치 매개변수는 무작위 초깃값인 상태)에서 문장을 생성
from ch07.rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
# model.load_params('ch06/Rnnlm.pkl')

# 시작(start) 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문자 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt) # 모델의 가중치 초깃값으로 무작위한 값을 사용하여 의미 없는 문장이 생성됨

Done
you two-thirds tapped lowering gang milestones legislator cypress guarantees nothing employee associations tapes probability repeat heart change roles help annualized deep felt jackets inspection colors charges restrictions driven mcalpine nor tv observed released estimates dropping sale laser philosophy retinoblastoma gates congressional class-action external attracting looming exploded journal supposed replacing grounds exports producing metropolitan dismal analyze midland greenhouse signals nsc seng newsprint monetary salesman reconciliation created crunch dynamic apparently hear manufactured resolutions true unraveled gains desert stepped itself somewhat disposable sole compelling worrisome april routes capitalized too 20th pitched national gifts paying formal easing devices together violations inception implications needed recognizes


In [7]:
# 위에서 구현한 RnnlmGen 클래스를 사용하여 문장 생성
# 학습을 통해 나온 가중치를 불러와 학습을 수행
from ch07.rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('ch06/Rnnlm.pkl')

# 시작(start) 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문자 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt) # 첫번째로 수행한 결과보다는 괜찮은 결과가 생성되었지만 개선할 여지가 존재

you can better execution of medicine.
 it will continue to be impossible for legislation to market.
 mr. roman declined to appear the outlook on the conceptual attack and other changes falling.
 mr. bush is to loose talks to projects he will take weeks.
 domestic democrats were misleading by the press 's veto done.
 gop plans to place from accomplished between authority and takeover-stock cities.
 mr. gelbart 's executive r. hunt was president of the state attorney general 's prestigious director of the staff ' institute.
 mr. anderson was involved into boren who president


### 더 좋은 문장으로 생성하기 위한 개선방안
- 언어 모델을 향상시켜 더 좋은 문장을 생성하도록 개선 r

In [8]:
# BetterRnnlm 클래스를 상속받아 BetterRnnlmGen 클래스 생성
import numpy as np

from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class BetterRnnlmGen(BetterRnnlm) : # Rnnlm을 상속받음

  # 문장을 생성하는 메서드
  # ========== 입력 파라미터 ========== 
  # start_id - 최초로 주는 단어의 ID
  # sample_size - 샘플링하는 단어의 수
  # skip_ids - 단어 Id의 리스트로 이에 속한 단어 ID는 샘플링하지 않음. PTB에 있는 <unk>나 전처리된 단어를 샘플링하지 않는 용도
  def generate(self, start_id, skip_ids=None, sample_size=100) :
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size : 
      x = np.array(x).reshape(1,1) # model.predict()에서 미니배치 처리하므로 2차원 배열로 성형(reshape)
      score = self.predict(x) # 각 단어의 정규화되기 전 점수(score) 출력
      p = softmax(score.flatten()) # 소프트맥스 함수로 정규화하여 확률분포 P를 얻음

      sampled = np.radnom.choice(len(p), size=1, p=p) # 확률 분포를 바탕으로 다음 단어 샘플링
      if (skip_ids is None) or (sampled not in skip_ids) :
        x = sampled
        word_ids.append(int(x))

    return word_ids


In [ ]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = BetterRnnlmGen()
model.load_params('ch07/BetterRnnlm.pkl')

# 시작(start) 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문자 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt) 

- 위에서 생성한 언어 모델에게 'the meaning of life is'라는 글을 주고, 이어지는 말을 생성하도록 구현도 가능
- 모델에 ['the', 'meaning', 'of', 'life']를 순서대로 주어 순전파를 수행하고, 그 다음 'is'를 첫 단어로 입력하여 문장을 생성하면 이에 이어지는 문장 생성 가능


## seq2seq
- 언어 데이터, 음성 데이터, 동영상 데이터 모두 시계열 데이터
- 시계열 데이터를 다른 시계열 데이터로 변환하는 기법

### seq2seq의 원리
- Encoder-Decoder 모델이라고도 불리는데, 2개의 모듈인 인코더와 디코더가 등장
  - Encoder : 입력 데이터를 부호화(인코딩)

  * 부호화 : 정보를 어떤 규칙에 따라 변환하는 것

  - Decoder : 인코딩된 데이터를 디코딩(복호화)

  * 복호화 : 인코딩된 정보를 원래의 정보로 되돌리는 것

- 한국어를 영어로 번역하는 것 역시 이러한 과정을 통해 가능
  - 인코더 : '나는 고양이로소이다'를 출발어 문장으로 인코딩
  - 인코딩한 정보를 디코더에 전달
  - 디코더 : 도착어 문장을 생성하되 인코더가 인코딩한 정보에는 번역에 필요한 정보가 조밀하게 응축되어 있어 이를 바탕으로 디코더가 도착어 문장을 생성

- 이러한 과정을 RNN을 통해서 구현 가능

#### Encoder의 처리
- 인코더는 RNN을 사용하여 시계열 데이터(출발어 문장)를 h라는 은닉 상태 벡터로 변환
  - 이 과정을 RNN의 LSTM을 사용하거나 단순 RNN이나 GRU 등도 사용 가능

- LSTM 계층의 마지막 은닉 상태는 인코더가 출력하는 벡터 h로 입력문장을 번역하는데 필요한 정보가 인코딩 됨
- 이때,  h는 고정 길이 벡터이므로 인코딩은 곧 임의 길이의 문장을 고정 길이의 벡터로 변환하는 작업이라고 볼 수 있음

### 시계열 데이터 변환용 장난감 문제
- 시계열 변환 문제의 예로 '더하기'를 살펴봄
- '57+5'를 문자열로 seq2seq로 건네면 62라는 정답을 내도록 학습

* 참고 : 머신러닝을 평가하고자 만든 간단한 문제를 '장난감 문제(toy problem)'이라고 함

- seq2seq로 덧셈을 풀기위해서는 샘플로부터 사용되는 문자의 패턴을 학습하여 규칙을 학습해야 함
- 이 문제를 해결하기 위하여 단어 단위가 아닌 문자 단위로 분할
  - 문자 단위 분할 : 문자별로 리스트 처리

### 가변 길이 시계열 데이터 
- 이러한 덧셈 문제는 샘플마다 데이터의 시간 방향 크기가 다름
  - 예) 628+521 이 들어올지 57+1이 들어올 지 모름
  - 즉, 가변 길이 시계열 데이터를 다룬다는 의미이므로 신경망 학습 시 '미니배치 처리'를 위한 노력이 필요함

- 가변 길이 시계열 데이터를 미니배치로 학습하기 위해 사용할 수 있는 가장 단순한 방법은 패딩(padding)을 사용하는 것
  - 패딩 : 원래의 데이터에 의미 없는 데이터를 채워서 모든 데이터의 길이를 균일하게 맞출 수 있는 기법

- 패딩을 적용하여 데이터 크기를 통일시키면 가변 길이 시계열 데이터도 처리가능하지만 존재하지 않던 문자까지도 seq2seq가처리하게 됨
  - 따라서 seq2seq에 패딩 전용 처리도 추가해야함
  - 디코더에 입력된 데이터가 패딩일 경우 손실의 결과에 반영하지 않도록.
  - 인코더에 입력된 데이터가 패딩일 경우 LSTM 계층이 이전 시각의 입력을 그대로 출력하도록 하여 처리 가능

### 덧셈 데이터셋을 사용

In [14]:
from dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

print(x_train[0])
print(t_train[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)
[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]
71+118 
_189 
